In [1]:
import numpy as np
import pandas as pd
import matplotlib
import geopandas as gpd
import datetime,os,re,sys,subprocess
from socket import gethostname
from tabulate import tabulate
import h5py
from simpledbf import Dbf5 as dbf

In [2]:
file = "./shapefiles/Node/FREEFLOW_nodes.dbf"
file_dbf = dbf(file)
df_node = file_dbf.to_dataframe()
df_node.head()

,N,X,Y,DTA_EDIT_F,MULTI_NODE,PROJ
0,1,6.008120e+06,2.086465e+06,0,0,NaN
1,2,5.993658e+06,2.086661e+06,0,0,NaN
2,3,6.006012e+06,2.086611e+06,0,0,NaN
3,4,5.995632e+06,2.086719e+06,0,0,NaN
4,5,6.007266e+06,2.086606e+06,0,0,NaN


In [3]:
df_node.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23046 entries, 0 to 23045
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   N           23046 non-null  int64  
 1   X           23046 non-null  float64
 2   Y           23046 non-null  float64
 3   DTA_EDIT_F  23046 non-null  int64  
 4   MULTI_NODE  23046 non-null  int64  
 5   PROJ        207 non-null    object 
dtypes: float64(2), int64(3), object(1)
memory usage: 1.1+ MB


In [46]:
def readEqvFile(eqvfile):
	""" Reads the given eqvfile and returns
        distnames: distnum -> distname
		distToTaz: distnum -> list of taznums
		tazToDist: taznum  -> list of distnums
		numdists:  just the number of districts
	"""		
	f = open(eqvfile, 'r')
	eqvtxt = f.read()
	f.close()

	distline_re	= re.compile('DIST (\d+)=(\d+)( .+)?')
	lines 		= eqvtxt.split("\n")
	lineno 		= 0
	distnames	= {}
	distToTaz	= {}
	tazToDist	= {} 
	while (lineno < len(lines)):
		m 		= distline_re.search(lines[lineno])
		if (m != None):
			# distnames[int(m.group(1))] = m.group(2)
			dist= int(m.group(1))
			taz = int(m.group(2))
			if (dist not in distToTaz):
				distToTaz[dist] = []              
			distToTaz[dist].append(taz)
			if (taz not in tazToDist):
				tazToDist[taz] = []
			tazToDist[taz].append(dist)
			if (m.group(3) != None):
				distnames[dist] = m.group(3).strip(' ')
		lineno	= lineno + 1
	numdists	= len(distnames)
	return (distnames, distToTaz, tazToDist, numdists) 

def readSummitSumFile(sumfile, tablekeys, numdists):
    """ Reads the given summit sumfile and returns
    sumnums: tablekey -> 2d list of number strings (numdist x numdist)
    """
    f 			= open(sumfile)
    sumtxt 		= f.read()
    f.close()

    lines		= sumtxt.split("\n")
    sumnums		= {}
    
    for key in tablekeys:
        sumnums[key] = []
        start = (key-1)*(numdists+1)+1
        # print(start)
        for lineno in range(start,start+numdists):
            sumnums[key].append(lines[lineno].split("|"))
    return sumnums

def scaleSummitNums(sumnums):
	""" Scale the unscaled and make everything a number.  Return floats.
	"""
	for key in sumnums.keys():
		for i1 in range(0,len(sumnums[key])):
			for i2 in range(0,len(sumnums[key][i1])):
				sumnums[key][i1][i2] = float(sumnums[key][i1][i2])

	return sumnums

def sumRowAndCol(squarelist, row, col, includeIntersection=True, cumulative=True):
    """ What it sounds like
    """
    sum = 0
    if (cumulative == False):
        intersection = squarelist[row][col] 
        # print(len(squarelist))
        for r in range(0, len(squarelist)):
            sum = sum + squarelist[r][col]
        for c in range(0, len(squarelist)):
            sum = sum + squarelist[row][c]
        # we double counted this one
        sum = sum - intersection
        if (includeIntersection == False):
            sum = sum - intersection
    else:
        sum = 0
        # rows first
        for r in range(0, row+1):
            for c in range(0, len(squarelist)):
                sum = sum + squarelist[r][c]
        # cols
        for c in range(0, col+1):
            for r in range(row+1, len(squarelist)):
                sum = sum + squarelist[r][c]
    return sum      

In [47]:
eqv = 'shapefiles/DIST15.eqv'
(distnames, distToTaz, tazToDist, numdists) = readEqvFile(eqv)

In [48]:
def summitToModeSumToOrFrom(sumnums, numdists, runtype, within=True, timePeriod='Daily'):
    """ Normalizes summit output to Mode share numbers for to/from (and optionally within)
        runtype:    'champ' or 'RPM9'
        within:     True for To,From or Within; False for just To or From
            Returns
        timePeriod: "Daily", "AM", "MD", "PM", "EV", or "EA" (this is from the summit ctl)
        modesum: Auto|Transit|Pedestrian|Bike|Total -> list of trips for each dist for that mode
    """
    modesum = {}
    for mode in MS_ROWS[runtype]:
        modesum[mode] = []
    base = -1 
    # base = 0
    if (timePeriod=='Daily'):   base = 0
    elif (timePeriod=='AM'):    base = 10
    elif (timePeriod=='MD'):    base = 20
    elif (timePeriod=='PM'):    base = 30
    elif (timePeriod=='EV'):    base = 40
    elif (timePeriod=='EA'):    base = 50
    else: 
        print( "Don't understand timePeriod: "+timePeriod)
        return modesum
    if (runtype == "RPM9"):
        for dist in range(0, numdists): 
            # TO NOte we have changed the cummulative as False to not have a cummulative sum for districts
            # Auto, Auto toll, & Auto Paid
            modesum[MS_AUTO].append( \
                sumRowAndCol(sumnums[base+1],dist,dist,within, False) + \
                sumRowAndCol(sumnums[base+2],dist,dist,within, False) + \
                sumRowAndCol(sumnums[base+3],dist,dist,within, False))
            # Walk-To-Transit & Drivebase+-To-Transit
            modesum[MS_TRANSIT].append( \
                sumRowAndCol(sumnums[base+6],dist,dist,within, False) + \
                sumRowAndCol(sumnums[base+7],dist,dist,within, False))
            modesum[MS_PED].append( \
                sumRowAndCol(sumnums[base+4],dist,dist,within, False))
            modesum[MS_BIKE].append( \
                sumRowAndCol(sumnums[base+5],dist,dist,within, False))
            modesum['TNC'].append( \
                sumRowAndCol(sumnums[base+9],dist,dist,within, False) + \
                sumRowAndCol(sumnums[base+10],dist,dist,within, False))
    elif (runtype == "champ"):
        for dist in range(0, numdists): 
            modesum[MS_AUTO].append( \
                sumRowAndCol(sumnums[base+1],dist,dist,within, True)) 
            modesum[MS_TRANSIT].append( \
                sumRowAndCol(sumnums[base+2],dist,dist,within, True))
            modesum[MS_PED].append( \
                sumRowAndCol(sumnums[base+3],dist,dist,within, True))
            modesum[MS_BIKE].append( \
                sumRowAndCol(sumnums[base+4],dist,dist,within, True))
    else:
        print ("Could not calculate mode share for unknown runtype " + runtype)
    return modesum

def modeSumToModeShare(modesum):
    """ modesum numbers -> percentages for each dist
    """
    total       = []
    modeshare   = {}
    for mode in modesum.keys():
        modeshare[mode] = []    
    for dist in range(0,len(modesum[mode])):
        total.append(0)
        for mode in modesum.keys():
            modeshare[mode].append(0.0)
            total[dist] = total[dist] + modesum[mode][dist]
        for mode in modesum.keys():
            num = round(100 * float(modesum[mode][dist]) / float(total[dist]),1)
            modeshare[mode][dist] = num
    # print total
    return modeshare

In [5]:
ftables = {"ftable1": 'data/PERSONTRIPS_AM.h5',
           "ftable2": 'data/PERSONTRIPS_MD.h5',
           "ftable3": 'data/PERSONTRIPS_PM.h5',
           "ftable4": 'data/PERSONTRIPS_EV.h5',
           "ftable5": 'data/PERSONTRIPS_EA.h5'}

tables = {}
# store all tables in the "tables" dictionary 
for ftable, filename in ftables.items():
    with h5py.File(filename, "r") as f:
        print(f.keys())
        for name in f.keys():
            table = f[name][...] # get all the dataset
            # so the first table in "ftable1 would be ftable11"
            tables[ftable + name] = table

<KeysViewHDF5 ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']>
<KeysViewHDF5 ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']>
<KeysViewHDF5 ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']>
<KeysViewHDF5 ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']>
<KeysViewHDF5 ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '3', '4', '5', '6', '7', '8', '9']>


In [6]:
tables.keys()

dict_keys(['ftable11', 'ftable110', 'ftable111', 'ftable112', 'ftable113', 'ftable114', 'ftable115', 'ftable116', 'ftable117', 'ftable118', 'ftable119', 'ftable12', 'ftable120', 'ftable121', 'ftable13', 'ftable14', 'ftable15', 'ftable16', 'ftable17', 'ftable18', 'ftable19', 'ftable21', 'ftable210', 'ftable211', 'ftable212', 'ftable213', 'ftable214', 'ftable215', 'ftable216', 'ftable217', 'ftable218', 'ftable219', 'ftable22', 'ftable220', 'ftable221', 'ftable23', 'ftable24', 'ftable25', 'ftable26', 'ftable27', 'ftable28', 'ftable29', 'ftable31', 'ftable310', 'ftable311', 'ftable312', 'ftable313', 'ftable314', 'ftable315', 'ftable316', 'ftable317', 'ftable318', 'ftable319', 'ftable32', 'ftable320', 'ftable321', 'ftable33', 'ftable34', 'ftable35', 'ftable36', 'ftable37', 'ftable38', 'ftable39', 'ftable41', 'ftable410', 'ftable411', 'ftable412', 'ftable413', 'ftable414', 'ftable415', 'ftable416', 'ftable417', 'ftable418', 'ftable419', 'ftable42', 'ftable420', 'ftable421', 'ftable43', 'ftab

In [7]:
#Change the range for different mode mapping
for i in range(1, 6):
    #1-9 are Auto, Transit, Pedestrian, Bike tables
    for j in range(1, 9):
        name = "t{}{}".format(i, j)
        value = tables["ftable{}{}".format(i, j)]
        #store all tables in the local dictionary, 
        # so the variable t11 would be the first table in "ftable1"
        locals()[name] = value
    #19, 20 are TNC tables
    for j in range(19,21):
        name = "t{}{}".format(i, j)
        value = tables["ftable{}{}".format(i, j)]
        locals()[name] = value

#sum them up   
t1 = t11 + t21 + t31 + t41 + t51
t2 = t12 + t22 + t32 + t42 + t52
t3 = t13 + t23 + t33 + t43 + t53
t4 = t14 + t24 + t34 + t44 + t54
t5 = t15 + t25 + t35 + t45 + t55
t6 = t16 + t26 + t36 + t46 + t56
t7 = t17 + t27 + t37 + t47 + t57
t8 = t18 + t28 + t38 + t48 + t58
t9 = t119 + t219 + t319 + t419 + t519
t10 = t120 + t220 + t320 + t420 + t520


eqvfile = 'shapefiles/DIST15.eqv'
# eqvfile = 'shapefiles/DISTSIMPLE3.eqv'
distnames, distToTaz, tazToDist, numdists = readEqvFile(eqvfile)

In [26]:
t11

array([[0.22, 0.  , 0.28, ..., 0.04, 0.  , 0.  ],
       [0.  , 0.  , 0.72, ..., 0.05, 0.  , 0.  ],
       [0.28, 0.  , 0.  , ..., 0.05, 0.  , 0.  ],
       ...,
       [0.04, 0.05, 0.04, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [8]:
distToTaz.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [9]:
comb_arr=[]
for j in range(1,numdists+1):
    comb_arr.append([i-1 for i in distToTaz[j]])

In [59]:
# comb_arr = District as index with tazs in value

In [35]:

for i in range(len(comb_arr)):
    for j in comb_arr[i]:
        if j==0:
            print(i,j)

2314
8 0


In [10]:
flatten_comb_arr = [item for sublist in comb_arr for item in sublist]

In [11]:
MS_AUTO                 = 'Auto'
MS_TRANSIT              = 'Transit'
MS_PED                  = 'Pedestrian'
MS_BIKE                 = 'Bike'
MS_TNC                  =  "TNC"
MS_ROWS                 = { 'RPM9':[ MS_AUTO, MS_TRANSIT, MS_PED, MS_BIKE, MS_TNC ],
                            'champ':[ MS_AUTO, MS_TRANSIT, MS_PED, MS_BIKE, MS_TNC ] }

In [12]:
def readSummitSumFile1(sumfile, tablekeys, numdists):
    """ Reads the given summit sumfile and returns
    sumnums: tablekey -> 2d list of number strings (numdist x numdist)
    """
    f 			= open(sumfile)
    sumtxt 		= f.read()
    f.close()

    lines		= sumtxt.split("\n")
    sumnums		= {}
    
    for key in tablekeys:
        sumnums[key] = []
        start = (key-1)*(numdists+1)+1
        # print(start)
        for lineno in range(start,start+numdists):
            # print(lineno)
            sumnums[key].append(lines)
    return sumnums

In [13]:
def readSumFile(sumfile):
    f = open(sumfile)
    text = f.read()
    f.close()
    lines = text.split("\n")
    # print(lines)
    sumnums={}
    typ = 0
    for i, line in enumerate(lines):
        if line=='':
            typ+=1
            sumnums[typ]=[]
            continue
        sumnums[typ].append(line)
        

In [193]:
output = [ ]
flatten_comb_arr = [item for sublist in comb_arr for item in sublist]
for t in [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10]:
    res = []
    for taz in flatten_comb_arr:
        res.append(int(round(t[taz].sum())))
    output.append(res)

print(len(flatten_comb_arr)) #2453
# 59*2314+1
# Columns are taz and index are type of mode
taz_df = pd.DataFrame(data=output,index=range(1,11),columns=flatten_comb_arr)
# taz_df = taz_df.apply(lambda col: 100*col/sum(col))
taz_df=taz_df.T
taz_df.index = taz_df.index + 1

2314


In [199]:
# flatten_comb_arr
taz_df.loc[582]

1     3
2     0
3     0
4     1
5     0
6     1
7     0
8     5
9     1
10    1
Name: 582, dtype: int64

In [128]:
len(output) #t1 = t11 + t21 + t31 + t41 + t51
tables['ftable21'][580].sum()
# t1[580].sum()
# np.count_nonzero(tables['ftable11'][580])


2.2199999999999998

In [202]:
T_taz_df = taz_df.reset_index(names='Taz')
mode_taz = pd.DataFrame()
mode_taz['Auto'] = T_taz_df.iloc[:,1:4].sum(axis=1)
mode_taz['Transit'] = T_taz_df.iloc[:,6:8].sum(axis=1)
mode_taz['Pedestrian'] = T_taz_df.iloc[:,4:5].sum(axis=1)
mode_taz['Bike'] = T_taz_df.iloc[:,5:6].sum(axis=1)
mode_taz['TNC'] = T_taz_df.iloc[:,9:11].sum(axis=1)

mode_taz = mode_taz.apply(lambda col: 100*col/sum(col),axis=1)
mode_taz['Taz'] = T_taz_df['Taz']
mode_taz

,Auto,Transit,Pedestrian,Bike,TNC,Taz
0,31.724441,26.099166,20.122861,2.790698,19.262835,286
1,27.886710,20.221826,21.142801,2.327194,28.421470,296
2,32.516590,16.865748,24.430832,3.103624,23.083206,299
3,29.920303,17.776577,23.260840,2.269350,26.772930,300
4,25.584795,17.056530,22.831384,2.777778,31.749513,302
...,...,...,...,...,...,...
2309,91.605712,0.000000,5.242076,1.706722,1.445489,2400
2310,89.634319,0.000000,6.708873,1.077199,2.579609,2401
2311,76.021768,0.000000,19.480231,2.561824,1.936177,2402
2312,80.398804,0.000000,17.078764,1.296112,1.226321,2403


In [204]:
mode_taz[mode_taz['Taz']==313]
# T_taz_df

,Auto,Transit,Pedestrian,Bike,TNC,Taz
109,NaN,NaN,NaN,NaN,NaN,313


In [205]:
mode_taz['District'] = mode_taz['Taz'].map(tazToDist)
mode_taz['District'] = mode_taz['District'].str[0]
# mode_taz['District'].fillna(value=0,inplace=True)
# mode_taz['District'] = mode_taz['District'].astype(int)
mode_taz.to_csv('taz_mode.csv',index=False)
# tazToDist[mode_taz['Taz']][0]

In [213]:
mode_taz[mode_taz['Auto'].isnull()]

,Auto,Transit,Pedestrian,Bike,TNC,Taz,District
109,NaN,NaN,NaN,NaN,NaN,313,2
110,NaN,NaN,NaN,NaN,NaN,385,2
175,NaN,NaN,NaN,NaN,NaN,872,2
210,NaN,NaN,NaN,NaN,NaN,384,3
889,NaN,NaN,NaN,NaN,NaN,305,12
...,...,...,...,...,...,...,...
1046,NaN,NaN,NaN,NaN,NaN,1065,13
1047,NaN,NaN,NaN,NaN,NaN,1066,13
1048,NaN,NaN,NaN,NaN,NaN,1067,13
1049,NaN,NaN,NaN,NaN,NaN,1068,13


In [214]:
len(comb_arr
)

15

In [253]:
# comb_arr

In [49]:
#combine all the result into a large list
output = [ ]
for t in [t1,t2,t3,t4,t5,t6,t7,t8,t9,t10]:
    res = []
    for start in comb_arr:
        row = []
        for end in comb_arr:
            row.append(t[start][:,end].sum())
        res.append(row)
    output.append(res)
    
table_list = [str(a)+str(b) for a in range(1,6) for b in [i for i in range(1,10)]+[20]]
for x in table_list:
    res = []
    for start in comb_arr:
        row = []
        for end in comb_arr:
            row.append(tables[f"ftable{x}"][start][:,end].sum())
        res.append(row)
    output.append(res)

    
#generate the summary sum file like the "summit" application did
with open(f'summit_file.sum', 'w') as file:
    file.write("\n")
    for res in output:
        for row in res:
            file.write('|'.join(map(str, row)) + '\n')
        file.write("\n")



tablekeys = [i for i in range(1,61)]


sumnums = readSummitSumFile(f'summit_file.sum', tablekeys, numdists)

# Scale the summit numbers
sumnums = scaleSummitNums(sumnums)

# Define time periods of interest
timePeriods = ['Daily','AM','MD','PM','EV','EA']

# Compute mode shares for each time period
res = {}
modesum_ck={}
percentage_dict = dict()
for time in timePeriods:
    # Convert summit data to mode sums and then to mode shares
    # print(time)
    modesum = summitToModeSumToOrFrom(sumnums, numdists, 'RPM9', True, timePeriod=time)
    res[time] = modeSumToModeShare(modesum)
    modesum_ck[time] = modesum
    total_mode_sum = dict()
    for key in modesum.keys():
        total_mode_sum[key]=np.sum(modesum[key][0:12])
    sumt=0
    for val in total_mode_sum.values():
        sumt+=val
    percentage_dict[time] = {key: round((value / sumt) * 100,2) for key, value in total_mode_sum.items()}

# Convert the mode shares to strings with a percent sign for printing to CSV
# for time in res.keys():
#     for key in res[time].keys():
#         res[time][key] = [f"{i}%" for i in res[time][key]]

# Define lists of places and transportation types of interest
places = list(distnames.values())
types = [MS_AUTO, MS_TRANSIT, MS_PED, MS_BIKE, MS_TNC]

In [50]:
excel_writer = pd.ExcelWriter('Output_mode_withboth.xlsx', engine='xlsxwriter')
# pd.DataFrame(data = modesum_ck['Daily']).to_excel(excel_writer, sheet_name='Daily', index=True)

for time in timePeriods:
    pd.DataFrame(data = modesum_ck[time]).to_excel(excel_writer, sheet_name=time, index=True)
excel_writer.close()


In [249]:
#Checking for Daily
modesum = summitToModeSumToOrFrom(sumnums, numdists, 'RPM9', True, timePeriod='EA')
# res[time] = modeSumToModeShare(modesum)
modesum_ck[time] = modesum
total_mode_sum = dict()
for key in modesum.keys():
    total_mode_sum[key]=np.sum(modesum[key][0:12])
sumt=0
for val in total_mode_sum.values():
    sumt+=val
# percentage_dict[time] = {key: round((value / sumt) * 100,2) for key, value in total_mode_sum.items()}


In [43]:
sumnums

{1: [[33335.0,
   12057.0,
   30127.0,
   14772.0,
   16773.0,
   4263.0,
   16327.0,
   8493.0,
   10183.0,
   3853.0,
   2580.0,
   7235.0,
   18754.0,
   27665.0,
   6677.0],
  [12065.0,
   9830.0,
   5706.0,
   7221.0,
   14430.0,
   3294.0,
   4917.0,
   2811.0,
   5693.0,
   2965.0,
   1851.0,
   4003.0,
   14035.0,
   18727.0,
   3068.0],
  [29813.0,
   5568.0,
   17841.0,
   6991.0,
   8158.0,
   1740.0,
   11589.0,
   4104.0,
   4048.0,
   1489.0,
   1084.0,
   3270.0,
   8851.0,
   13095.0,
   4456.0],
  [14005.0,
   6787.0,
   6819.0,
   23534.0,
   17429.0,
   6316.0,
   14000.0,
   9480.0,
   5192.0,
   4677.0,
   5209.0,
   13046.0,
   15431.0,
   11927.0,
   4499.0],
  [17099.0,
   14329.0,
   8081.0,
   17334.0,
   46625.0,
   13532.0,
   9282.0,
   5524.0,
   17658.0,
   8980.0,
   6068.0,
   9946.0,
   29830.0,
   21612.0,
   3793.0],
  [4038.0,
   3169.0,
   1731.0,
   6319.0,
   13441.0,
   10201.0,
   2848.0,
   1839.0,
   9445.0,
   7521.0,
   5507.0,
   6342.0,
 

In [250]:
total_mode_sum #EA

{'Auto': 416448.0,
 'Transit': 315585.0,
 'Pedestrian': 76704.0,
 'Bike': 13797.0,
 'TNC': 30533.0}

In [244]:
modesum_ck['Daily']['Auto'],modesum_ck['EA']['Auto']

([389534.0,
  575456.0,
  730191.0,
  967910.0,
  1267546.0,
  1388262.0,
  1526189.0,
  1650349.0,
  1837945.0,
  2004194.0,
  2092596.0,
  2297111.0,
  9389377.0,
  16611274.0,
  18633817.0],
 [8690.0,
  13903.0,
  18011.0,
  23589.0,
  31841.0,
  34505.0,
  38575.0,
  41701.0,
  46555.0,
  49952.0,
  51905.0,
  57221.0,
  208347.0,
  354348.0,
  390423.0])

In [16]:
pec = 100*mode_auto/np.sum(modesum['Auto'])
pec[:12].sum()

NameError: name 'mode_auto' is not defined

In [ ]:
mode_auto=np.array(modesum['Auto'])
100*mode_auto

In [17]:
total_mode_sum = dict()
percentage_dict = dict()
for key in modesum.keys():
    total_mode_sum[key]=np.sum(modesum[key][0:12])
sumt=0
for val in total_mode_sum.values():
    sumt+=val
percentage_dict[time] = {key: round((value / sumt) * 100,2) for key, value in total_mode_sum.items()}

In [251]:
# res

In [342]:
# df = pd.DataFrame(data=res['Daily'], columns=types, index=places)
df_alt = df.loc[:,['District','Auto','Transit','TNC','Bike','Pedestrian']]
df_alt.index.name = 'District No'
markdown_table = tabulate(df_alt, headers='keys', tablefmt='pipe').split('\n')
markdown_table[0] = '| ' + ' | '.join(f'**{header.strip()}**' for header in markdown_table[0].split('|')[1:-1]) + ' |'
markdown_table[1] = markdown_table[1].replace('|', ':|:').replace('::', ':')[1:-1]
markdown_table = '\n'.join(markdown_table)
print(markdown_table)
with open(f'Mode_dist_daily.md', 'w') as f:
    f.write(markdown_table)

| **District No** | **District** | **Auto** | **Transit** | **TNC** | **Bike** | **Pedestrian** |
|:--------------:|:-------------------:|:-------:|:----------:|:------:|:-------:|:-------------:|
|             1 | Downtown           |   28.2 |      34.4 |  19.5 |    2.3 |         15.6 |
|             2 | SoMa               |   32.3 |      31.9 |  19.2 |    2.3 |         14.3 |
|             3 | N.Beach/ Chinatown |   34   |      30.4 |  19.1 |    2.3 |         14.2 |
|             4 | Western Market     |   38   |      27.7 |  17.4 |    2.4 |         14.5 |
|             5 | Mission/ Potrero   |   42.4 |      25.6 |  15.7 |    2.4 |         13.9 |
|             6 | Noe/ Glen/ Bernal  |   44   |      24.8 |  15   |    2.4 |         13.7 |
|             7 | Marina/ N.Heights  |   45.4 |      24   |  14.5 |    2.4 |         13.7 |
|             8 | Richmond           |   46.7 |      23.3 |  13.9 |    2.4 |         13.6 |
|             9 | Bayshore           |   48.8 |      22.2 |  13.2 |

In [325]:
distnames.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [341]:
df = pd.DataFrame(data=res['Daily'], columns=types, index=places)
df = df.reset_index()
df.index = distnames.keys()
df.columns = ['District','Auto','Transit','Pedestrian','Bike','TNC']
df

,District,Auto,Transit,Pedestrian,Bike,TNC
1,Downtown,28.2,34.4,15.6,2.3,19.5
2,SoMa,32.3,31.9,14.3,2.3,19.2
3,N.Beach/ Chinatown,34.0,30.4,14.2,2.3,19.1
4,Western Market,38.0,27.7,14.5,2.4,17.4
5,Mission/ Potrero,42.4,25.6,13.9,2.4,15.7
6,Noe/ Glen/ Bernal,44.0,24.8,13.7,2.4,15.0
7,Marina/ N.Heights,45.4,24.0,13.7,2.4,14.5
8,Richmond,46.7,23.3,13.6,2.4,13.9
9,Bayshore,48.8,22.2,13.4,2.4,13.2
10,Outer Mission,50.2,21.4,13.3,2.4,12.6


In [229]:
df1 = pd.DataFrame(data=percentage_dict['AM'], columns=types, index=[1])
df2 = pd.DataFrame(data=percentage_dict['MD'], columns=types, index=places)
df3 = pd.DataFrame(data=percentage_dict['PM'], columns=types, index=places)
df4 = pd.DataFrame(data=percentage_dict['EV'], columns=types, index=places)
df5 = pd.DataFrame(data=percentage_dict['EA'], columns=types, index=places)

In [236]:
df1
percentage_dict

{'Daily': {'Auto': 45.04,
  'Transit': 24.28,
  'Pedestrian': 13.71,
  'Bike': 2.4,
  'TNC': 14.57},
 'AM': {'Auto': 46.4,
  'Transit': 33.94,
  'Pedestrian': 11.06,
  'Bike': 2.77,
  'TNC': 5.84},
 'MD': {'Auto': 47.59,
  'Transit': 20.65,
  'Pedestrian': 17.84,
  'Bike': 2.49,
  'TNC': 11.43},
 'PM': {'Auto': 46.95,
  'Transit': 29.29,
  'Pedestrian': 13.18,
  'Bike': 2.65,
  'TNC': 7.94},
 'EV': {'Auto': 52.1,
  'Transit': 22.7,
  'Pedestrian': 13.94,
  'Bike': 2.51,
  'TNC': 8.75},
 'EA': {'Auto': 48.82,
  'Transit': 36.99,
  'Pedestrian': 8.99,
  'Bike': 1.62,
  'TNC': 3.58}}

In [316]:
df = pd.DataFrame(data=percentage_dict).T
df.reset_index().to_csv(f"Mode_tod.csv",index=False)

In [347]:
df.reset_index().to_csv(f"district_mode_tod.csv",index=False)

In [319]:
df_alt = df.loc[:,['Auto','Transit','TNC','Bike','Pedestrian']]

In [234]:
mean_df = pd.DataFrame()

for df in [df1, df2, df3, df4, df5]:
    mean = df.mean(axis=0)
    mean = mean.round(2)
    mean_df = pd.concat([mean_df, mean], axis=1, ignore_index=True)

mean_df.columns = types
mean_df.index = ['AM', 'MD', 'PM', 'EV', 'EA']
mean_df.reset_index().to_csv(f"Mode_tod.csv",index=False)

In [320]:
df_alt.index.name = 'TOD'
markdown_table = tabulate(df_alt, headers='keys', tablefmt='pipe').split('\n')
markdown_table[0] = '| ' + ' | '.join(f'**{header.strip()}**' for header in markdown_table[0].split('|')[1:-1]) + ' |'
markdown_table[1] = markdown_table[1].replace('|', ':|:').replace('::', ':')[1:-1]
markdown_table = '\n'.join(markdown_table)
print(markdown_table)
with open(f'Mode_tod.md', 'w') as f:
    f.write(markdown_table)

| **TOD** | **Auto** | **Transit** | **TNC** | **Bike** | **Pedestrian** |
|:------:|:-------:|:----------:|:------:|:-------:|:-------------:|
| Daily |  45.04 |     24.28 | 14.57 |   2.4  |        13.71 |
| AM    |  46.4  |     33.94 |  5.84 |   2.77 |        11.06 |
| MD    |  47.59 |     20.65 | 11.43 |   2.49 |        17.84 |
| PM    |  46.95 |     29.29 |  7.94 |   2.65 |        13.18 |
| EV    |  52.1  |     22.7  |  8.75 |   2.51 |        13.94 |
| EA    |  48.82 |     36.99 |  3.58 |   1.62 |         8.99 |


In [187]:
df = pd.DataFrame(data=res['Daily'], columns=types, index=places)
df.reset_index().to_csv('Mode_daily.csv',index=False)

# PANDAS implementation of mode share

In [282]:
ftables = {"ftable1": 'data/PERSONTRIPS_AM.h5',
           "ftable2": 'data/PERSONTRIPS_MD.h5',
           "ftable3": 'data/PERSONTRIPS_PM.h5',
           "ftable4": 'data/PERSONTRIPS_EV.h5',
           "ftable5": 'data/PERSONTRIPS_EA.h5'}

tables = {}
# store all tables in the "tables" dictionary 
for ftable, filename in ftables.items():
    with h5py.File(filename, "r") as f:
        for name in f.keys():
            table = f[name][...] # get all the dataset
            # so the first table in "ftable1 would be ftable11"
            tables[ftable + name] = table

In [283]:
tables.keys()

dict_keys(['ftable11', 'ftable110', 'ftable111', 'ftable112', 'ftable113', 'ftable114', 'ftable115', 'ftable116', 'ftable117', 'ftable118', 'ftable119', 'ftable12', 'ftable120', 'ftable121', 'ftable13', 'ftable14', 'ftable15', 'ftable16', 'ftable17', 'ftable18', 'ftable19', 'ftable21', 'ftable210', 'ftable211', 'ftable212', 'ftable213', 'ftable214', 'ftable215', 'ftable216', 'ftable217', 'ftable218', 'ftable219', 'ftable22', 'ftable220', 'ftable221', 'ftable23', 'ftable24', 'ftable25', 'ftable26', 'ftable27', 'ftable28', 'ftable29', 'ftable31', 'ftable310', 'ftable311', 'ftable312', 'ftable313', 'ftable314', 'ftable315', 'ftable316', 'ftable317', 'ftable318', 'ftable319', 'ftable32', 'ftable320', 'ftable321', 'ftable33', 'ftable34', 'ftable35', 'ftable36', 'ftable37', 'ftable38', 'ftable39', 'ftable41', 'ftable410', 'ftable411', 'ftable412', 'ftable413', 'ftable414', 'ftable415', 'ftable416', 'ftable417', 'ftable418', 'ftable419', 'ftable42', 'ftable420', 'ftable421', 'ftable43', 'ftab

In [286]:
ft11 = pd.DataFrame(data=tables['ftable11'])
ft11

,0,1,2,3,4,5,6,7,8,9,...,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474
0,0.22,0.00,0.28,0.81,0.81,4.91,2.81,2.34,0.16,0.13,...,0.0,0.03,0.00,0.0,0.0,0.04,0.0,0.04,0.0,0.0
1,0.00,0.00,0.72,0.56,0.00,0.28,0.88,2.56,0.59,1.34,...,0.0,0.03,0.00,0.0,0.0,0.04,0.0,0.05,0.0,0.0
2,0.28,0.00,0.00,0.06,1.00,1.34,2.72,4.25,1.00,0.75,...,0.0,0.03,0.00,0.0,0.0,0.04,0.0,0.05,0.0,0.0
3,0.00,0.84,0.03,2.38,1.13,0.31,1.03,1.03,4.63,0.41,...,0.0,0.05,0.01,0.0,0.0,0.06,0.0,0.07,0.0,0.0
4,0.81,0.50,1.91,0.06,1.38,1.84,0.91,1.72,2.03,0.09,...,0.0,0.02,0.00,0.0,0.0,0.04,0.0,0.04,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,0.03,0.04,0.04,0.06,0.03,0.15,0.01,0.06,0.10,0.09,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0
2471,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.01,0.01,0.01,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0
2472,0.04,0.05,0.04,0.08,0.04,0.22,0.01,0.08,0.15,0.12,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0
2473,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.01,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0
